In [1]:
# My OpenAI Key
import os
import openai
import sys
if 'think_aloud' in sys.modules:
    del sys.modules['think_aloud']

from think_aloud import *

openai.api_key = os.getenv("OPENAI_API_KEY")

## Processing Website Data

In [2]:
websites = ["Expedia.com", "Booking.com", "Trip.com"]
tasks = ["Task 1: search for a round-trip flight from Toronto to New York, departing on February 1 2024, and returning on February 15, 2024. Find the cheapest option that has < 2 layovers and total travel time < 8 hours",
         "Task 2: search for a hotel in Manhattan, New York City for the dates of their flight. The hotel should have Wi-Fi and breakfast included, be rated at least four stars, and price should not exceed $300 per night",
         "Task 3: search for a rental car, pickup and dropoff at JFK, for the dates of their flight. The car should be a standard sized sedan, have unlimited mileage, and price < $80 per day."]
metrics = """
            "Task Name","Mental Demand (Scale 1-7)","Physical Demand (Scale 1-7)","Perceived Task Pace (Scale 1-7)","Task Success Perception (Scale 1-7)","Effort Level to Achieve Performance (Scale 1-7)","Emotional Stress Level (Scale 1-7)","Number of Fixations Overall","Saccade to Fixation Ratio","Number of Saccades","Task Completion Time (s)","Time Until First Click (s)","Mean Time Between Clicks (s)","Number of Mouse Clicks","Scanpath Length (px)","Mousepath Length (px)"
            "Expedia Task 1",2,"1 (Low)",2,5,2,4,10761,"18.2%",1959,224,2,6,32,268919.02,15229.22
            "Expedia Task 2",2,"1 (Low)",2,5,2,2,8981,"19.9%",1783,209,13,10,19,233192.33,8266.11
            "Expedia Task 3","1 (Low)","1 (Low)","1 (Low)","1 (Perfect)","1 (Low)","1 (Low)",5659,"21.2%",1197,119,0,5,23,155385.34,10839.55
            "Kayak Task 1",2,"1 (Low)","1 (Low)","1 (Perfect)","1 (Low)","1 (Low)",8765,"17.6%",1540,179,1,8,22,215285.41,10092.51
            "Kayak Task 2","1 (Low)","1 (Low)",2,"7 (Failure)",3,4,9393,"20.1%",1885,209,3,9,22,231106.77,10946.90
            "Kayak Task 3","1 (Low)","1 (Low)","1 (Low)",4,5,6,10740,"22.1%",2369,224,0,3,66,259267.74,18672.11
            """
audio_files = ["audio.mp3", "audio2.mp3"]
logs = ["expedia_logs.txt", "kayak_logs.txt"]

## Processing Think Aloud Data

In [ ]:
transcriptions = transcribe_audio(audio_files)
print(transcriptions[0]['text'])
print(transcriptions[1]['text'])

In [27]:
minutes = [think_aloud_minutes(t['text']) for t in transcriptions]
minutes = [format_transcript(transcript, idx) for idx, transcript in enumerate(minutes)]
print(minutes[0])
print(minutes[1])

 Okay, so I'm going to look for New York for the dates on 1st of November until the 30th November. It's just one traveler and yeah, I chose I accidentally searched for hotels instead of searching for flights. So I'm just going to go back even from Toronto going to New York. Same dates, 1st of October, 1th of November. So New York and the cheapest option is already sorted by price. The cheapest one is over $299. I said it's twice the other conditions which are less than $12 hour time and it has zero layovers. I'm perfectly turned flight the cheapest option also satisfies the conditions. Easy easy. Next I'm going to go. Now I'm going to search for hotels. Going to New York. I'm going to search for the first 30th one traveler. Search. Oh, I'm happy to make a free box for the appointment. I'm going to change that. The location setting. The hotel should have a breakfast included. Free Wi-Fi. Wi-Fi included. I second to filter it. We rated at least four stars. I'm going to choose the first o

In [33]:

insights = [summarize_insights(minute) for minute in minutes]
insights = [str(i+1) + ". " + websites[i] + ": " + insights[i] for i in range(len(insights))]
print(insights[0])
print(insights[1])

["1. Expedia.com: Key Insights and Common Themes:\n\n1. Clarity and Intuitiveness: Users often struggle with unclear or non-intuitive search and filter options. This includes difficulty in differentiating between search options, finding specific filters, and understanding the functionality of each filter. \n\n2. Default Settings: Users prefer certain default settings, such as sorting by price, which the website currently does not provide. \n\n3. Inclusion of Common Amenities: Users want an easy way to include common amenities in their search criteria, suggesting that these are important factors in their decision-making process.\n\n4. Budgeting: Users want an easy way to set a budget for their purchases, indicating that cost is a significant consideration for them.\n\n5. Performance: Users experience delays while the website loads, suggesting that the website's performance could be improved.\n\n6. Total Cost Display: Users want to see the total cost of their purchases over a selected pe

## Process Logs

In [34]:
contents = []

for log in logs:
    with open(log, 'r') as file:
        content = file.read()
        contents.append(content)

# Now, contents[0] has the content of expedia.txt as a string
# and contents[1] has the content of kayak.txt as a string


analyses = [process_log(content) for content in contents]
analyses = [str(i+1) + ". " + websites[i] + ": " + analyses[i] for i in range(len(analyses))]
print(analyses[0])
print(analyses[1])

## Final Processing

In [37]:
query = f"""
You are given the results of a user study. This study was designed to evaluate the usability of the following websites: {websites}

The participant was instructed to conduct tasks on each website. The tasks were as follows:
{tasks}

Metrics generated from study: {metrics}

Insights from think-aloud data: {insights}

Insights from browser logs: {analyses}

Let’s first understand the problem and devise a plan to conduct the comprehensive analysis. Then, carry out the plan and conduct the analysis step by step.
"""

print(query)



You are given the results of a user study. This study was designed to evaluate the usability of the following websites: ['Expedia.com', 'Kayak.com']

The participant was instructed to conduct tasks on each website. The tasks were as follows:
['Task 1: search for a round-trip flight from Toronto to New York, departing on December 20, 2023, and returning on January 10, 2024. Find the cheapest option that has < 2 layovers and total travel time < 8 hours', 'Task 2: search for a hotel in Manhattan, New York City for the dates of their flight. The hotel should have Wi-Fi and breakfast included, be rated at least four stars, and price should not exceed $250 per night', 'Task 3: search for a rental car, pickup and dropoff at JFK, for the dates of their flight. The car should be a standard sized sedan, have unlimited mileage, and price < $60 per day.']

Metrics generated from study: 
            "Task Name","Mental Demand (Scale 1-7)","Physical Demand (Scale 1-7)","Perceived Task Pace (Scale 1

In [38]:
response = openai.ChatCompletion.create(
    model="gpt-4",
    temperature=0,
    messages=[
        {
            "role": "system",
            "content": "You are uxGPT, an AI ux researcher that uses a combination of eye-tracking and click-tracking metrics, usability behavioral metrics, and web page data from usability studies to identify usability issues and provide actionable insights."
        },
        {
            "role": "user",
            "content": query
        },
    ],
)['choices'][0]['message']['content']

print(response)

Understanding the Problem:

The problem at hand is to analyze the usability of two travel booking websites, Expedia.com and Kayak.com, based on the results of a user study. The user study involved participants performing specific tasks on each website, and metrics were collected to evaluate the user experience. The goal is to identify usability issues and provide actionable insights for improvement.

Plan for Analysis:

1. Analyze the Metrics: Review the metrics generated from the study for each task on both websites. Look for patterns and anomalies that might indicate usability issues.

2. Analyze the Think-Aloud Data: Review the insights from the think-aloud data to understand the user's thought process and struggles while performing the tasks.

3. Analyze the Browser Logs: Review the insights from the browser logs to understand the user's behavior and identify potential areas of confusion or trouble spots.

4. Synthesize Findings: Combine the insights from the metrics, think-aloud d

In [39]:
response2 = openai.ChatCompletion.create(
    model="gpt-4",
    temperature=0,
    messages=[
        {
            "role": "system",
            "content": "You are uxGPT, an AI ux researcher that uses a combination of eye-tracking and click-tracking metrics, usability behavioral metrics, and web page data from usability studies to identify usability issues and provide actionable insights."
        },
        {
            "role": "user",
            "content": query
        },
        {
            "role": "system",
            "content": response
        },
        {
            "role": "user",
            "content": "Continue."
        }
    ],
)['choices'][0]['message']['content']

print(response2)

Continuing with the analysis, let's delve deeper into the specific issues and recommendations for each website:

Expedia.com:

Issues:

1. Unclear or Non-Intuitive Search and Filter Options: Users struggled to differentiate between search options and find specific filters. This could lead to frustration and a longer time to complete tasks.

2. Lack of Preferred Default Settings: Users preferred certain default settings, such as sorting by price, which the website currently does not provide. This could lead to users feeling that the website is not tailored to their needs.

3. Difficulty Including Common Amenities: Users found it difficult to include common amenities in their search criteria. This could lead to users not finding the options they want and potentially choosing another website.

4. Performance Issues: Users experienced delays while the website loaded. This could lead to frustration and users leaving the website.

5. Lack of Total Cost Display: Users wanted to see the total 

In [40]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
print(response)


Understanding the Problem:

The problem at hand is to analyze the usability of two travel booking websites, Expedia.com and Kayak.com, based on the results of a user study. The user study involved participants performing specific tasks on each website, and metrics were collected to evaluate the user experience. The goal is to identify usability issues and provide actionable insights for improvement.

Plan for Analysis:

1. Analyze the Metrics: Review the metrics generated from the study for each task on both websites. Look for patterns and anomalies that might indicate usability issues.

2. Analyze the Think-Aloud Data: Review the insights from the think-aloud data to understand the user's thought process and struggles while performing the tasks.

3. Analyze the Browser Logs: Review the insights from the browser logs to understand the user's behavior and identify potential areas of confusion or trouble spots.

4. Synthesize Findings: Combine the insights from the metrics, think-aloud d

In [41]:
print(analyses[0])

1. Expedia.com: 1. Patterns of User Behavior:
   - The user seems to be planning a trip, as they are searching for flights and hotels in New York City and Toronto.
   - The user is using the search function extensively, indicating they are actively looking for specific information.
   - The user is interacting with various elements such as buttons, inputs, and dropdowns, indicating they are trying to customize their search or booking.
   - The user is hovering over various elements before clicking, suggesting they are reading the information carefully before making a selection.
   - The user is repeatedly returning to the homepage, possibly to start a new search or to navigate to a different section of the website.

2. Deviations from Typical Behavior Indicating Usability Issues:
   - The user seems to be having trouble with the date selection for flights and hotels, as they hover and click on these elements multiple times.
   - The user is clicking on the same buttons multiple times, 

In [43]:
print(response2)

Continuing with the analysis, let's delve deeper into the specific issues and recommendations for each website:

Expedia.com:

Issues:

1. Unclear or Non-Intuitive Search and Filter Options: Users struggled to differentiate between search options and find specific filters. This could lead to frustration and a longer time to complete tasks.

2. Lack of Preferred Default Settings: Users preferred certain default settings, such as sorting by price, which the website currently does not provide. This could lead to users feeling that the website is not tailored to their needs.

3. Difficulty Including Common Amenities: Users found it difficult to include common amenities in their search criteria. This could lead to users not finding the options they want and potentially choosing another website.

4. Performance Issues: Users experienced delays while the website loaded. This could lead to frustration and users leaving the website.

5. Lack of Total Cost Display: Users wanted to see the total 

In [2]:
response3 = openai.ChatCompletion.create(
    model="gpt-4-1106-preview",
    temperature=0,
    messages=[
        {
            "role": "system",
            "content": "You are The UX Insight Assistant is designed to assist in the creation of the Oculyze AI system and its associated web app. It analyzes usability study data, such as think-aloud, eye gaze, mouse tracking, and click event data, to generate insightful reports on website usability. The assistant also helps in web app design, guiding website owners to conduct usability studies and interpret results. It communicates in a clear, accessible language, suitable for a broad audience, including those without deep technical knowledge in UX research. This approach ensures practical, understandable advice for designing effective and user-friendly websites, aligning with the project's goal for submission to UIST 2024."
        },
        {
            "role": "user",
            "content": """
            We need to create a prototype web app for this tool. Users input the wesbites, tasks to be completed, and upload the eye + click tracking data (.tsv) and browser logs (.txt) for each task on each website, as well as some relevant screenshots of each website and one audio file of think aloud data for each website. I already have a python script that taken in these information and generates a usability report.




Next, we need to build on this. Modify the web app to include separate fields. The user selects the number of **participants** in the study (let that be x), the number of **websites** (let it be y), and the number of **tasks** per website (let that be z). they also upload some screenshots of each website. They also need to specify the website names  and provide short descriptions of each task. 

Then, for each participant they need to upload the following:
1. Think aloud files for each website (y files per participant, audio)
2. eye + click tracking data for each task on each website (yz  files per participant, .tsv)
3. browser logs for each task on each website (yz files per participant, .txt)

Web app user should upload each file separately. 

I have already updated index.html:
```
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>UX Study Input</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
    <script src="{{ url_for('static', filename='script.js') }}"></script>
</head>
<body>
    <h1>UX Study Input</h1>
    <form id="studyInputForm" method="post" enctype="multipart/form-data">
        <label for="participants">Number of Participants:</label>
        <input type="number" id="participants" name="participants" min="1" required><br><br>

        <label for="websites">Number of Websites:</label>
        <input type="number" id="websites" name="websites" min="1" required><br><br>

        <label for="tasksPerWebsite">Number of Tasks per Website:</label>
        <input type="number" id="tasksPerWebsite" name="tasksPerWebsite" min="1" required><br><br>

        <div id="dynamicInputs"></div>

        <input type="submit" value="Submit">
    </form>
</body>
</html>
```

script.js:
```
window.onload = function() {
    const form = document.getElementById('studyInputForm');
    
    form.onchange = function(e) {
        const participants = document.getElementById('participants').value;
        const websites = document.getElementById('websites').value;
        const tasksPerWebsite = document.getElementById('tasksPerWebsite').value;

        let dynamicInputs = document.getElementById('dynamicInputs');
        dynamicInputs.innerHTML = '';

        for (let i = 1; i <= participants; i++) {
            for (let j = 1; j <= websites; j++) {
                // Website Name and Screenshot Upload
                dynamicInputs.innerHTML += `<h3>Participant ${i}, Website ${j}</h3>`;
                dynamicInputs.innerHTML += `<input type='text' name='websiteName_${i}_${j}' placeholder='Website Name' required>`;
                dynamicInputs.innerHTML += `<input type='file' name='screenshot_${i}_${j}' accept='image/*'><br>`;

                for (let k = 1; k <= tasksPerWebsite; k++) {
                    // Task Description, Think Aloud, Eye+Click Data, Browser Log
                    dynamicInputs.innerHTML += `<h4>Task ${k}</h4>`;
                    dynamicInputs.innerHTML += `<input type='text' name='taskDesc_${i}_${j}_${k}' placeholder='Task Description' required>`;
                    dynamicInputs.innerHTML += `<input type='file' name='thinkAloud_${i}_${j}_${k}' accept='audio/*'>`;
                    dynamicInputs.innerHTML += `<input type='file' name='eyeClickData_${i}_${j}_${k}' accept='.tsv'>`;
                    dynamicInputs.innerHTML += `<input type='file' name='browserLog_${i}_${j}_${k}' accept='.txt'><br>`;
                }
            }
        }
    };
};
```

app.py:
```
from flask import Flask, render_template, request, redirect, url_for

app = Flask(__name__)

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/upload', methods=['POST'])
def upload_files():
    form_data = request.form
    uploaded_files = request.files
    
    for key in uploaded_files:
        file = uploaded_files[key]
        if file.filename != '':
            file.save(os.path.join('uploads', file.filename))  # Save file to 'uploads' folder

    # Process the uploaded files and form data as needed

    return redirect(url_for('results'))


@app.route('/results')
def results():
    # Placeholder for results page
    return "Results will be displayed here."

if __name__ == '__main__':
    app.run(debug=True)

```



This code asks user to input website names and task descriptions for each participant. but websites and tasks are the same for all. How to fix?

First ask for number of websites and number of tasks, dynamically displaying fields to input website names and task descriptions. Below that, ask for number of participants, and dynamically display fields to upload files."
        """},
        #{
        #     "role": "system",
        #     "content": response
        # },
        # {
        #     "role": "user",
        #     "content": "Continue."
        # }
    ],
)['choices'][0]['message']['content']

response3

'To address the issue you\'ve described, we need to restructure the form to first collect the website names and task descriptions, which are common to all participants, and then collect the participant-specific files. Here\'s how you can modify your `index.html` and `script.js` to achieve this:\n\n### index.html\n```html\n<!-- ... existing code ... -->\n<div id="websiteAndTaskInputs"></div> <!-- Placeholder for website and task inputs -->\n\n<label for="participants">Number of Participants:</label>\n<input type="number" id="participants" name="participants" min="1" required><br><br>\n\n<div id="participantInputs"></div> <!-- Placeholder for participant-specific inputs -->\n\n<input type="submit" value="Submit">\n<!-- ... existing code ... -->\n```\n\n### script.js\n```javascript\nwindow.onload = function() {\n    const form = document.getElementById(\'studyInputForm\');\n    const participantsInput = document.getElementById(\'participants\');\n    const websitesInput = document.getElem